In [3]:
import matplotlib.pyplot as plt
import os
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web
from datetime import datetime as dt
import pickle

warnings.simplefilter(action='ignore', category=FutureWarning)

In [176]:
def get_multi_factor_model_data():
    START_DATE = '1954-01-01'
    # three factors 
    df_three_factor = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=START_DATE)[0]
    df_three_factor.index = df_three_factor.index.format()

    # momentum factor
    df_mom = web.DataReader('F-F_Momentum_Factor', 'famafrench', start=START_DATE)[0]
    df_mom.index = df_mom.index.format()

    # three factors
    df_four_factor = df_three_factor.join(df_mom)

    # five factors
    df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start=START_DATE)[0]
    df_five_factor.index = df_five_factor.index.format()

    df_three_factor.to_csv('data/multi-factor_models/F-F_Research_Data_Factors.csv')
    df_four_factor.to_csv('data/multi-factor_models/Carhart_4_Factors.csv')
    df_five_factor.to_csv('data/multi-factor_models/F-F_Research_Data_5_Factors_2x3.csv')

get_multi_factor_model_data()

In [7]:
def get_multi_factor_data():
    f = 'data/multi-factor_models'
    df_three_factor = pd.read_csv(os.path.join(f, 'F-F_Research_Data_Factors.csv'), 
                                  index_col=0)
    df_four_factor = pd.read_csv(os.path.join(f, 'Carhart_4_Factors.csv'), 
                                  index_col=0)
    df_five_factor = pd.read_csv(os.path.join(f, 'F-F_Research_Data_5_Factors_2x3.csv'), 
                                  index_col=0)

    return df_three_factor, df_four_factor, df_five_factor


df_three_factor, df_four_factor, df_five_factor = get_multi_factor_data()                               

In [10]:
START_DATE = '2019-01-01'
END_DATE = '2022-07-01'

In [4]:
SPY_info_df = pd.read_csv('data/SPY-Info.csv')
ticker_list = SPY_info_df['Symbol'].to_list()

In [8]:
def get_ticker_data(ticker):
    f = 'data/market_data'
    ticker_df = pd.read_csv(os.path.join(f, f'{ticker}.csv'), index_col=0,
                            parse_dates=True)

    return ticker_df

In [9]:
def get_model_parameters(model): 
    params = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
    params = params.rename(columns={'P>|t|': 'p-value'})
    f_pvalue = round(model.f_pvalue, 3)
    rsquared = round(model.rsquared, 3)
    rsquared_adj = round(model.rsquared_adj, 3)
    
    return params, f_pvalue, rsquared, rsquared_adj

# params, f_pvalue, rsquared, rsquared_adj = get_model_parameters(five_factor_model)


In [11]:
def make_factor_model(model, start, end, ticker):
    ticker_df = get_ticker_data(ticker)
    y = ticker_df['adjclose'].resample('M') \
                             .last() \
                             .pct_change() \
                             .dropna()
    y.index = y.index.strftime('%Y-%m')
    y.name = 'return'

    if model == 3:
        model_df = df_three_factor
        model_data = model_df.join(y, how='inner')
        model_data.columns = ['mkt', 'smb', 'hml', 'rf', 'rtn']
        formula = 'excess_rtn ~ mkt + smb + hml'
    elif model == 4:
        model_df = df_four_factor
        model_data = model_df.join(y, how='inner')
        model_data.columns = ['mkt', 'smb', 'hml', 'rf', 'mom', 'rtn']
        formula = 'excess_rtn ~ mkt + smb + hml + mom'
    elif model == 5:
        model_df = df_five_factor
        model_data = model_df.join(y, how='inner')
        model_data.columns = ['mkt', 'smb', 'hml', 'rmw', 'cma', 'rf', 'rtn']
        formula = 'excess_rtn ~ mkt + smb + hml + rmw + cma'

    model_data.loc[:, model_data.columns != 'rtn'] /= 100
    model_data.index = [pd.to_datetime(x, format='%Y-%m') for x in model_data.index]
    model_data = model_data.loc[start:end]
    model_data['excess_rtn'] = model_data.rtn - model_data.rf

    if not model_data.empty:
        factor_model = smf.ols(formula=formula, data=model_data).fit()
        params, f_pvalue, rsquared, rsquared_adj = get_model_parameters(factor_model)
    else:
        params, f_pvalue, rsquared, rsquared_adj = [pd.DataFrame()] + [np.nan] * 3
    
    return params, f_pvalue, rsquared, rsquared_adj

In [12]:
def make_ticker_factor_models():
    relevant_ticker_factor_models = {}
    irrelevant_ticker_factor_models = {}
    for model in [3, 4, 5]:
        ticker_factor_models = {}
        relevant_factor_models = {}
        irrelevant_factor_models = {}
    
        for ticker in ticker_list:
            params, f_pvalue, rsquared, rsquared_adj = make_factor_model(model, START_DATE, END_DATE, ticker)
            ticker_factor_models[ticker] = {'f_pvalue': f_pvalue, 
                                            'rsquared_adj': rsquared_adj, 
                                            'params': params}
            if params.empty:
                irrelevant_factor_models[ticker] =  ticker_factor_models[ticker]
            else:
                if f_pvalue <= 0.05 and not all(x > 0.05 for x in params['p-value']) \
                    and (params.loc['Intercept', 'p-value'] > 0.05 or model == 3):
                    relevant_factor_models[ticker] =  ticker_factor_models[ticker]
                else:
                    irrelevant_factor_models[ticker] =  ticker_factor_models[ticker]
        
        relevant_ticker_factor_models[f'{model}_factor_models'] = relevant_factor_models
        irrelevant_ticker_factor_models[f'{model}_factor_models'] = irrelevant_factor_models

        print(f"{len(relevant_factor_models) / len(ticker_list) * 100:.0f}% " +
              f"of stock returns explained by {model}-factor model")

    f = r'data\multi-factor_models'

    with open(os.path.join(f, f'relevant_ticker_factor_models.pickle'), 'wb') as file:
        pickle.dump(relevant_ticker_factor_models, file)

    with open(os.path.join(f, f'irrelevant_ticker_factor_models.pickle'), 'wb') as file:
        pickle.dump(irrelevant_ticker_factor_models, file)

make_ticker_factor_models()

c:\Users\pnjor\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\Users\pnjor\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


94% of stock returns explained by 3-factor model


c:\Users\pnjor\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\Users\pnjor\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


89% of stock returns explained by 4-factor model


c:\Users\pnjor\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\Users\pnjor\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\Users\pnjor\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\Users\pnjor\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
c:\Users\pnjor\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuin

90% of stock returns explained by 5-factor model


Compare predictions to actual results

In [5]:
def get_ticker_factor_models():
    f = r'data\multi-factor_models'

    with open(os.path.join(f, 'relevant_ticker_factor_models.pickle'), 'rb') as file:
        relevant_ticker_factor_models = pickle.load(file)
    with open(os.path.join(f, 'irrelevant_ticker_factor_models.pickle'), 'rb') as file:
        irrelevant_ticker_factor_models = pickle.load(file)
    
    return relevant_ticker_factor_models, irrelevant_ticker_factor_models
    
relevant_ticker_factor_models, irrelevant_ticker_factor_models = get_ticker_factor_models()

In [6]:
relevant_3_factor_models = relevant_ticker_factor_models['3_factor_models']
relevant_4_factor_models = relevant_ticker_factor_models['4_factor_models']
relevant_5_factor_models = relevant_ticker_factor_models['5_factor_models']
irrelevant_3_factor_models = irrelevant_ticker_factor_models['3_factor_models']
irrelevant_4_factor_models = irrelevant_ticker_factor_models['4_factor_models']
irrelevant_5_factor_models = irrelevant_ticker_factor_models['5_factor_models']

In [13]:
columns = ['Ticker', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Adj_R2', 'Predicted', 'Actual', '|Actual-Predicted|']

In [14]:
relevant_3_factor_models

{'MMM': {'f_pvalue': 0.0,
  'rsquared_adj': 0.548,
  'params':              coef  std err      t  p-value  [0.025  0.975]
  Intercept -0.0137    0.007 -1.899    0.066  -0.028   0.001
  mkt        0.7937    0.138  5.734    0.000   0.512   1.075
  smb        0.2069    0.254  0.815    0.421  -0.309   0.723
  hml        0.1346    0.141  0.958    0.345  -0.151   0.421},
 'AOS': {'f_pvalue': 0.001,
  'rsquared_adj': 0.354,
  'params':              coef  std err      t  p-value  [0.025  0.975]
  Intercept  0.0011    0.012  0.090    0.929  -0.024   0.027
  mkt        1.0863    0.240  4.528    0.000   0.598   1.574
  smb       -0.1828    0.440 -0.416    0.680  -1.078   0.712
  hml       -0.0458    0.244 -0.188    0.852  -0.542   0.450},
 'ABT': {'f_pvalue': 0.0,
  'rsquared_adj': 0.395,
  'params':              coef  std err      t  p-value  [0.025  0.975]
  Intercept  0.0034    0.008  0.407    0.687  -0.014   0.020
  mkt        0.7692    0.161  4.773    0.000   0.441   1.097
  smb       -0.198

In [160]:
from datetime import timedelta

dt.strptime(END_DATE, '%Y-%m') + timedelta(days=31)

datetime.datetime(2022, 2, 1, 0, 0)

In [177]:
END_DATE = '2022-01'
ticker = 'GOOGL'
ticker_df = get_ticker_data(ticker)
y = ticker_df['adjclose'].resample('M') \
                            .last() \
                            .pct_change() \
                            .dropna()

df = df_four_factor
df.columns = df.columns.str.lower()
df.columns = df.columns.str.strip()
df = df.rename(columns={'mkt-rf': 'mkt'})
factors = df.loc[END_DATE, df_four_factor.columns != 'rf']
betas = relevant_4_factor_models[ticker]['params'].coef

# pred_date = dt.strptime(END_DATE, '%Y-%m') + timedelta(days=31)
y['2022-02']


2022-02-28   -0.001822
Freq: M, Name: adjclose, dtype: float64

In [175]:
relevant_4_factor_models[ticker]

{'f_pvalue': 0.0,
 'rsquared_adj': 0.496,
 'params':              coef  std err      t  p-value  [0.025  0.975]
 Intercept  0.0096    0.009  1.093    0.282  -0.008   0.027
 mkt        1.0416    0.189  5.513    0.000   0.657   1.426
 smb       -0.1963    0.318 -0.617    0.542  -0.845   0.452
 hml        0.1107    0.206  0.537    0.595  -0.309   0.530
 mom        0.1289    0.257  0.502    0.619  -0.394   0.652}

In [172]:
rtn = 0  # expected return according to model
for x in factors.index:
    # print(x, betas[x])
    rtn += factors[x] * betas[x]

rtn

-4.257873000000001

Testing effect of FCFF/share on returns

In [71]:
def get_annual_ticker_ratio(ticker, ratio):
    f = 'data/financial_ratios/Annual'
    df = pd.read_csv(os.path.join(f, f'{ticker}.csv'), index_col=0)
    df = df.drop(index='period').fillna(0).T
    df.index = [pd.to_datetime(x, format='%Y-%m-%d') for x in df.index]
    df = df[ratio].resample('Y').last().astype('float').sort_index()
    df.index = [pd.to_datetime(x, format='%Y-%m-%d') for x in df.index]

    return df

In [72]:
fcff = get_annual_ticker_ratio('AAPL', 'freeCashFlowPerShare')
fcff.index

DatetimeIndex(['1985-12-31', '1986-12-31', '1987-12-31', '1988-12-31',
               '1989-12-31', '1990-12-31', '1991-12-31', '1992-12-31',
               '1993-12-31', '1994-12-31', '1995-12-31', '1996-12-31',
               '1997-12-31', '1998-12-31', '1999-12-31', '2000-12-31',
               '2001-12-31', '2002-12-31', '2003-12-31', '2004-12-31',
               '2005-12-31', '2006-12-31', '2007-12-31', '2008-12-31',
               '2009-12-31', '2010-12-31', '2011-12-31', '2012-12-31',
               '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31',
               '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31',
               '2021-12-31'],
              dtype='datetime64[ns]', freq=None)

In [73]:
ticker_df = get_ticker_data('AAPL')
y = ticker_df.loc[:'2021', 'adjclose'].resample('Y') \
                                        .last() \
                                        .pct_change() \
                                        .dropna()
y.name = 'rtn'    
y.index = [pd.to_datetime(x, format='%Y-%m-%d') for x in y.index]
y

1981-12-31   -0.351652
1982-12-31    0.350292
1983-12-31   -0.184103
1984-12-31    0.194868
1985-12-31   -0.244636
1986-12-31    0.840919
1987-12-31    1.084447
1988-12-31   -0.033519
1989-12-31   -0.115624
1990-12-31    0.235010
1991-12-31    0.323313
1992-12-31    0.069115
1993-12-31   -0.504428
1994-12-31    0.352113
1995-12-31   -0.173362
1996-12-31   -0.345097
1997-12-31   -0.371255
1998-12-31    2.119031
1999-12-31    1.511454
2000-12-31   -0.710638
2001-12-31    0.472267
2002-12-31   -0.345661
2003-12-31    0.491276
2004-12-31    2.013571
2005-12-31    1.232609
2006-12-31    0.180137
2007-12-31    1.334748
2008-12-31   -0.569114
2009-12-31    1.469010
2010-12-31    0.530679
2011-12-31    0.255580
2012-12-31    0.325669
2013-12-31    0.080695
2014-12-31    0.406225
2015-12-31   -0.030137
2016-12-31    0.124804
2017-12-31    0.484643
2018-12-31   -0.053902
2019-12-31    0.889578
2020-12-31    0.823067
2021-12-31    0.346482
Name: rtn, dtype: float64

Get correlation

In [81]:
df = pd.DataFrame([fcff, y]).dropna(axis=1).T
df.head()

,freeCashFlowPerShare,rtn
1985-12-31,0.015219,-0.244636
1986-12-31,0.017213,0.840919
1987-12-31,0.006920,1.084447
1988-12-31,0.010311,-0.033519
1989-12-31,0.018596,-0.115624


In [78]:
df.corr?

Signature:
df.corr(
    method: 'str | Callable[[np.ndarray, np.ndarray], float]' = 'pearson',
    min_periods: 'int' = 1,
) -> 'DataFrame'
Docstring:
Compute pairwise correlation of columns, excluding NA/null values.

Parameters
----------
method : {'pearson', 'kendall', 'spearman'} or callable
    Method of correlation:

    * pearson : standard correlation coefficient
    * kendall : Kendall Tau correlation coefficient
    * spearman : Spearman rank correlation
    * callable: callable with input two 1d ndarrays
        and returning a float. Note that the returned matrix from corr
        will have 1 along the diagonals and will be symmetric
        regardless of the callable's behavior.
min_periods : int, optional
    Minimum number of observations required per pair of columns
    to have a valid result. Currently only available for Pearson
    and Spearman correlation.

Returns
-------
DataFrame
    Correlation matrix.

See Also
--------
DataFrame.corrwith : Compute pairwise corr